Compare speakers

In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
from src.spinorama.analysis import sound_power
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker, graph_melt
from src.spinorama.graph import graph_params_default

import datas.metadata as metadata
df = parse_all_speakers(metadata.speakers_info, None)
# print(df)

In [ ]:
df.keys()

In [ ]:
df['Adam S2V']['ASR']['default']['CEA2034'].head()

In [ ]:
def normalize(df):
    on = df[df.Measurements == 'On Axis']
    mean = np.mean(on.loc[(on.Freq>500) & (on.Freq<10000)].dB)
    
    offset = 0
    if 'DI offset' in df.Measurements.unique():
        offset = np.mean(df[df.Measurements == 'DI offset'].dB)
    else:
        offset = mean
        
    # print(mean, offset, df.Measurements.unique())
    # print(df[df.Measurements == 'Sound Power DI'].head())    
    for measurement in ('On Axis', 'Listening Window', 'Sound Power', 'Early Reflections'):
        df.loc[df.Measurements == measurement, 'dB'] -= mean
        
    for measurement in ('Sound Power DI', 'Early Reflections DI', 'DI offset'):
        df.loc[df.Measurements == measurement, 'dB'] -= offset
        
    return df

def pprint(df):
    for m in df.Measurements.unique():
        min = np.min(df[df.Measurements == m].dB)
        max = np.max(df[df.Measurements == m].dB)
        print('{0} {1} {2}'.format(min, max, m))
        
# adam = df['Adam S2V']['ASR']['default']['CEA2034']
# pprint(adam)
# norm_adam = normalize(adam)
# pprint(norm_adam)
# print(t[t.Measurements == 'Sound Power DI'].head()) 
# print(t[t.Measurements == 'On Axis'].head())

In [ ]:
def augment(dfa, name):
    # print(name)
    namearray = [name for i in range(0,len(dfa))]
    dfa['Speaker'] = name
    return dfa

source = pd.concat( 
    [augment(normalize(df[k][o]['default']['CEA2034']), k+' - '+o)
         for k in df.keys() for o in df[k].keys() if 'CEA2034' in df[k][o]['default']]
)

In [ ]:
source.shape
source.head()

In [ ]:
speaker1 = 'KEF LS50 - ASR'
speaker2 = 'KEF LS50 - Princeton'

nearest = alt.selection(
    type='single',
    nearest=True,
    on='mouseover',
    fields=['Freq'],
    empty='none')

def compare(df):
    speakers = df.Speaker.unique()
    input_dropdown1 = alt.binding_select(options=[s for s in speakers])
    selection1 = alt.selection_single(
        fields=['Speaker'],
        bind=input_dropdown1,
        name='Select right ',
        init={'Speaker': speaker1})
    input_dropdown2 = alt.binding_select(options=[s for s in speakers])
    selection2 = alt.selection_single(
        fields=['Speaker'],
        bind=input_dropdown2,
        name='Select left ',
        init={'Speaker': speaker2})

    line = alt.Chart(df).mark_line(
        clip=True,
        # strokeDash=[5,5]
    ).transform_filter(
        alt.FieldOneOfPredicate(
            field='Measurements',
            oneOf=['On Axis', 'Listening Window', 'Early Reflections', 'Sound Power'])
    ).encode(
        alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[20,20000], nice=False)),
        alt.Y('dB:Q',   scale=alt.Scale(zero=False, domain=[-40,10])),
        alt.Color('Measurements', type='nominal', sort=None)
    )
    
    di = alt.Chart(df).mark_line(clip=True).transform_filter(
        alt.FieldOneOfPredicate(
            field='Measurements',
            oneOf=['Early Reflections DI', 'Sound Power DI'])
    ).encode(
        alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[20,20000], nice=False)),
        alt.Y('dB:Q',   scale=alt.Scale(zero=False, domain=[0,30], nice=False)),
        alt.Color('Measurements', type='nominal', sort=None)
    )
    
    spin = (line+di).resolve_scale(y='independent').properties(width=600, height=300)

    line1 = spin.add_selection(selection1).transform_filter(selection1)
    line2 = spin.add_selection(selection2).transform_filter(selection2)

    #selectors = alt.Chart(df).mark_point().encode(x='Freq:Q', opacity=alt.value(0)).add_selection(nearest)                                               
    #points = line.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))                                                        
    #text = line.mark_text(align='left', dx=5, dy=-5).encode(text=alt.condition(nearest, 'dB:Q', alt.value(' ')))                                         
    #rules = alt.Chart(df).mark_rule(color='gray').encode(x='Freq:Q').transform_filter(nearest)                                                           
    center = alt.layer(line2,line1)# .interactive()
    return center

compare(source)